In [1]:
import json
import torch
import custom_utils
import numpy as np

import optuna

torch.set_default_device('cpu')

c:\Users\astus\code\extractive-summarization\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# importing data
with open('data/X_train.json', 'r') as json_file:
    X_train = json.load(json_file)
   
with open('data/y_train.json', 'r') as json_file:
    y_train = json.load(json_file)
   
with open('data/X_test.json', 'r') as json_file:
    X_test = json.load(json_file) 

with open('data/y_test.json', 'r') as json_file:
    y_test = json.load(json_file)
    
# converting to appropriate format (and device)
X_train = torch.Tensor(X_train).float()
y_train = torch.Tensor(y_train).long()

X_test = torch.Tensor(X_test).float()
y_test = torch.Tensor(y_test).long()

# getting shapes
num_features = X_train.shape[1]
num_classes = 2
# edge_dim = edge_attr.shape[1]

In [3]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from torchmetrics.classification import F1Score

In [4]:
def MLP(trial):
    # Model
    n_layers = trial.suggest_int('n_layers', 1, 4)
    layers = []

    in_features = num_features
    for i in range(n_layers):
        out_features = trial.suggest_int(f'n_units_l{i}', 200, 800)
        layers.append(torch.nn.Linear(in_features, out_features))
        layers.append(torch.nn.ReLU())
        
        # suggest dropout
        p = trial.suggest_float(f'p{i}', 0, 0.8)
        layers.append(torch.nn.Dropout(p))

        # updating next layer size
        in_features = out_features
        
    layers.append(torch.nn.Linear(in_features, num_classes))
    model = torch.nn.Sequential(*layers)
    return model

In [ ]:
# set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
def objective(trial):
    
    def train(X_train, y_train):
        model.train() 
        optimizer.zero_grad()
        out = model(X_train)
        loss = criterion(out, y_train)
        loss.backward()
        optimizer.step()
        return loss
    
    def validate(X_valid, y_valid):
        model.eval()
        out = model(X_valid)
        loss = criterion(out, y_valid)
        y_pred = out.argmax(dim=1)
        f1 = F1Score(task='binary', num_classes=num_classes).to(device)
        score = f1(y_valid, y_pred)
        # score = f1_score(y_valid, y_pred)
        return loss, score
        
    
    n_folds = 5
    n_epochs = 200
    patience = 10
    avg_score = 0
    skf = StratifiedKFold(n_splits=n_folds)
    for i, (train_idx, val_idx) in enumerate(skf.split(X_train,y_train)):
        
        # set model, criterion and optimizers
        model = MLP(trial).to(device)
        criterion = torch.nn.CrossEntropyLoss() # need to check input
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

        # Selecing fold train and validation
        train_mask = torch.zeros(len(X_train), dtype=torch.bool)
        val_mask = torch.zeros(len(X_train), dtype=torch.bool)
        train_mask[train_idx] = True
        val_mask[val_idx] = True
        
        X_train_fold = X_train[train_mask].to(device)
        y_train_fold = y_train[train_mask].to(device)
        X_valid_fold = X_train[val_mask].to(device)
        y_valid_fold = y_train[val_mask].to(device)
        
        # Epoch
        best_valid_loss = float('inf')
        score_at_best = -1
        current_patience = 0
        for epoch in range(n_epochs):
            train_loss = train(X_train_fold, y_train_fold)
            valid_loss, score = validate(X_valid_fold, y_valid_fold)
           
            # Stopping criterium            
            if valid_loss > best_valid_loss:
                current_patience += 1
            else:
                best_valid_loss = valid_loss
                score_at_best = score 
                current_patience = 0
            
            if current_patience == patience:
                break
            
            print(f'Epoch: {epoch}, Train loss: {train_loss:.4f}, Valid loss: {valid_loss:.4f}, Score: {score:.4f}')
        
        avg_score += score_at_best/n_folds
        
    return avg_score

In [6]:
# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2023-11-29 21:56:19,286] A new study created in memory with name: no-name-26debf4b-35de-4cbf-ab4a-7701aeb54d3b


Epoch: 0, Train loss: 0.7105, Valid loss: 0.5037, Score: 0.0000
Epoch: 1, Train loss: 0.5047, Valid loss: 0.4324, Score: 0.0000
Epoch: 2, Train loss: 0.4306, Valid loss: 0.4475, Score: 0.0000
Epoch: 3, Train loss: 0.4421, Valid loss: 0.4172, Score: 0.0000
Epoch: 4, Train loss: 0.4124, Valid loss: 0.3823, Score: 0.0000
Epoch: 5, Train loss: 0.3797, Valid loss: 0.3681, Score: 0.0000
Epoch: 6, Train loss: 0.3676, Valid loss: 0.3684, Score: 0.0000
Epoch: 7, Train loss: 0.3691, Valid loss: 0.3706, Score: 0.0000
Epoch: 8, Train loss: 0.3715, Valid loss: 0.3689, Score: 0.0000
Epoch: 9, Train loss: 0.3699, Valid loss: 0.3638, Score: 0.0000
Epoch: 10, Train loss: 0.3643, Valid loss: 0.3582, Score: 0.0000
Epoch: 11, Train loss: 0.3586, Valid loss: 0.3551, Score: 0.0000
Epoch: 12, Train loss: 0.3546, Valid loss: 0.3550, Score: 0.0000
Epoch: 13, Train loss: 0.3540, Valid loss: 0.3557, Score: 0.0000
Epoch: 14, Train loss: 0.3542, Valid loss: 0.3552, Score: 0.0000
Epoch: 15, Train loss: 0.3539, Vali

[W 2023-11-29 21:56:33,339] Trial 0 failed with parameters: {'n_layers': 1, 'n_units_l0': 368, 'p0': 0.20479142260877037} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\astus\code\extractive-summarization\.venv\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\astus\AppData\Local\Temp\ipykernel_13244\2734359479.py", line 55, in objective
    valid_loss, score = validate(X_valid_fold, y_valid_fold)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\astus\AppData\Local\Temp\ipykernel_13244\2734359479.py", line 17, in validate
    f1 = F1Score(task='binary', num_classes=num_classes).to(device)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\astus\code\extractive-summarization\.venv\Lib\site-packages\torch\nn\modules\module.py", line 1160, in to
    return self._appl

KeyboardInterrupt: 